<a href="https://colab.research.google.com/github/faheemshahid8017/DeepFake-Detector_AI_Project/blob/main/DeepFake_Detector_AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir templates
!mkdir uploads

In [3]:
%%writefile train.py
import os
import zipfile
import urllib3
import requests
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models  # type: ignore
from tensorflow.keras.layers import LeakyReLU  # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint  # type: ignore


class DatasetHandler:
    """
    A class to handle dataset downloading, unzipping, loading, and processing.
    """

    def __init__(self, dataset_url, dataset_download_dir, dataset_file, dataset_dir, train_dir, test_dir, val_dir):
        """
        Initialize the DatasetHandler with the specified parameters.

        Args:
            dataset_url (str): URL to download the dataset from.
            dataset_download_dir (str): Directory to download the dataset to.
            dataset_file (str): Name of the dataset file.
            dataset_dir (str): Directory containing the dataset.
            train_dir (str): Directory containing the training data.
            test_dir (str): Directory containing the test data.
            val_dir (str): Directory containing the validation data.
        """
        self.dataset_url = dataset_url
        self.dataset_download_dir = dataset_download_dir
        self.dataset_file = dataset_file
        self.dataset_dir = dataset_dir
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.val_dir = val_dir

    def download_dataset(self):
        """
        Download the dataset from the specified URL.

        Returns:
            bool: True if the dataset was successfully downloaded, False otherwise.
        """
        if not os.path.exists(self.dataset_download_dir):
            os.makedirs(self.dataset_download_dir)
        file_path = os.path.join(self.dataset_download_dir, self.dataset_file)
        if os.path.exists(file_path):
            print(f'dataset file {self.dataset_file} already exists at {file_path}')
            return True
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response = requests.get(self.dataset_url, stream=True, verify=False)
        total_size = int(response.headers.get('content-length', 0))
        with open(file_path, 'wb') as file, tqdm(desc=self.dataset_file, total=total_size, unit='iB', unit_scale=True, unit_divisor=1024) as bar:
            for data in response.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)
        print(f'dataset downloaded and saved to {file_path}')
        return True

    def unzip_dataset(self):
        """
        Unzip the downloaded dataset file.

        Returns:
            bool: True if the dataset was successfully unzipped, False otherwise.
        """
        file_path = os.path.join(self.dataset_download_dir, self.dataset_file)
        if os.path.exists(self.dataset_dir):
            print(f'dataset is already downloaded and extracted at {self.dataset_dir}')
            return True
        if not os.path.exists(file_path):
            print(f'dataset file {file_path} not found after download')
            return False
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(self.dataset_download_dir)
        print(f'dataset extracted to {self.dataset_dir}')
        return True

    def get_image_dataset_from_directory(self, dir_name):
        """
        Load image dataset from the specified directory.

        Args:
            dir_name (str): Name of the directory containing the dataset.

        Returns:
            tf.data.Dataset: Loaded image dataset.
        """
        dir_path = os.path.join(self.dataset_dir, dir_name)
        return tf.keras.utils.image_dataset_from_directory(
            dir_path,
            labels='inferred',
            color_mode='rgb',
            seed=42,
            batch_size=64,
            image_size=(128, 128)
        )

    def load_split_data(self):
        """
        Load and split the dataset into training, validation, and test datasets.

        Returns:
            tuple: Training, validation, and test datasets.
        """
        train_data = self.get_image_dataset_from_directory(self.train_dir)
        test_data = self.get_image_dataset_from_directory(self.test_dir)
        val_data = self.get_image_dataset_from_directory(self.val_dir)
        return train_data, test_data, val_data


class DeepfakeDetectorModel:
    """
    A class to create and train a deepfake detection model.
    """

    def __init__(self):
        """
        Initialize the DeepfakeDetectorModel by building the model.
        """
        self.model = self._build_model()

    def _build_model(self):
        """
        Build the deepfake detection model architecture.

        Returns:
            tf.keras.Model: Built model.
        """
        model = models.Sequential()
        model.add(layers.Input(shape=(128, 128, 3)))
        model.add(layers.Rescaling(1./127, name='rescaling'))
        model.add(layers.Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
        model.add(layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
        model.add(layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
        model.add(layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
        model.add(layers.Flatten())
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        return model

    def compile_model(self, learning_rate):
        """
        Compile the deepfake detection model.

        Args:
            learning_rate (float): Learning rate for the optimizer.
        """
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
        )

    def train_model(self, train_data, val_data, epochs):
        """
        Train the deepfake detection model.

        Args:
            train_data (tf.data.Dataset): Training dataset.
            val_data (tf.data.Dataset): Validation dataset.
            epochs (int): Number of epochs to train the model.

        Returns:
            tf.keras.callbacks.History: History object containing training details.
        """
        early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)
        model_checkpoint_callback = ModelCheckpoint('deepfake_detector_model_best.keras', monitor='val_loss', save_best_only=True, verbose=1)
        return self.model.fit(
            train_data,
            validation_data=val_data,
            epochs=epochs,
            callbacks=[early_stopping_callback, reduce_lr_callback, model_checkpoint_callback]
        )

    def evaluate_model(self, test_data):
        """
        Evaluate the deepfake detection model.

        Args:
            test_data (tf.data.Dataset): Test dataset.

        Returns:
            list: Evaluation metrics.
        """
        return self.model.evaluate(test_data)

    def save_model(self, path):
        """
        Save the deepfake detection model to the specified path.

        Args:
            path (str): Path to save the model.
        """
        self.model.save(path)


class TrainModel:
    """
    A class to manage training of a deepfake detection model.
    """

    def __init__(self, dataset_url, dataset_download_dir, dataset_file, dataset_dir, train_dir, test_dir, val_dir):
        """
        Initialize the TrainModel class with the specified parameters.

        Args:
            dataset_url (str): URL to download the dataset from.
            dataset_download_dir (str): Directory to download the dataset to.
            dataset_file (str): Name of the dataset file.
            dataset_dir (str): Directory containing the dataset.
            train_dir (str): Directory containing the training data.
            test_dir (str): Directory containing the test data.
            val_dir (str): Directory containing the validation data.
        """
        self.dataset_handler = DatasetHandler(dataset_url, dataset_download_dir, dataset_file, dataset_dir, train_dir, test_dir, val_dir)

    def run_training(self, learning_rate=0.0001, epochs=50):
        """
        Run the training process for the deepfake detection model.

        Args:
            learning_rate (float): Learning rate for the optimizer.
            epochs (int): Number of epochs to train the model.

        Returns:
            tuple: History object and evaluation metrics.
        """
        if not self.dataset_handler.download_dataset():
            print('failed to download dataset')
            return
        if not self.dataset_handler.unzip_dataset():
            print('failed to unzip dataset')
            return
        train_data, test_data, val_data = self.dataset_handler.load_split_data()
        model = DeepfakeDetectorModel()
        model.compile_model(learning_rate)
        history = model.train_model(train_data, val_data, epochs)
        evaluation_metrics = model.evaluate_model(test_data)
        model.save_model('deepfake_detector_model.keras')
        return history, evaluation_metrics


if __name__ == '__main__':
    # config
    dataset_url = 'https://www.kaggle.com/api/v1/datasets/download/manjilkarki/deepfake-and-real-images?datasetVersionNumber=1'
    dataset_download_dir = './data'
    dataset_file = 'dataset.zip'
    dataset_dir = './data/Dataset'
    train_dir = 'Train'
    test_dir = 'Test'
    val_dir = 'Validation'

    # instantiate the TrainModel class with the specified configuration
    trainer = TrainModel(
        dataset_url=dataset_url,
        dataset_download_dir=dataset_download_dir,
        dataset_file=dataset_file,
        dataset_dir=dataset_dir,
        train_dir=train_dir,
        test_dir=test_dir,
        val_dir=val_dir
    )

    # train
    history, evaluation_metrics = trainer.run_training(learning_rate=0.0001, epochs=50)

    # metrics
    print('evaluation metrics:', evaluation_metrics)

Writing train.py


In [4]:
%%writefile inference.py
from flask import Flask, request, render_template
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.preprocessing import image # type: ignore
import numpy as np
import os

class InferenceModel:
    """
    A class to load a trained model and handle file uploads for predictions.
    """

    def __init__(self, model_path):
        """
        Initialize the InferenceModel class.

        Args:
            model_path (str): Path to the saved Keras model.
        """
        self.model = load_model(model_path)
        self.app = Flask(__name__)
        self.app.config['UPLOAD_FOLDER'] = 'uploads'
        self.model_path = model_path

        @self.app.route('/', methods=['GET', 'POST'])
        def upload_file():
            """
            Handle file upload and prediction requests.

            Returns:
            --------
            str
                The rendered HTML template with the result or error message.
            """
            if request.method == 'POST':
                # check if the post request has the file part
                if 'file' not in request.files:
                    return render_template('index.html', error='no file part')
                file = request.files['file']
                # if user does not select file, browser also
                # submit an empty part without filename
                if file.filename == '':
                    return render_template('index.html', error='no selected file')
                if file and self.allowed_file(file.filename):
                    # save the uploaded file to the uploads directory
                    filename = os.path.join(self.app.config['UPLOAD_FOLDER'], file.filename)
                    file.save(filename)
                    # predict if the image is Real or Fake
                    prediction, prediction_percentage = self.predict_image(filename)
                    # clean up the uploaded file
                    os.remove(filename)
                    # determine result message
                    result = 'Fake' if prediction >= 0.5 else 'Real'
                    # render result to the user
                    return render_template('index.html', result=result, prediction_percentage=prediction_percentage)
                else:
                    return render_template('index.html', error='allowed file types are png, jpg, jpeg')
            return render_template('index.html')

    def allowed_file(self, filename):
        """
        Check if a file has an allowed extension.

        Parameters:
        -----------
        filename : str
            The name of the file to check.

        Returns:
        --------
        bool
            True if the file has an allowed extension, False otherwise.
        """
        ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
        return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

    def predict_image(self, file_path):
        """
        Predict whether an image is Real or Fake using the loaded model.

        Parameters:
        -----------
        file_path : str
            The path to the image file.

        Returns:
        --------
        tuple
            A tuple containing the prediction and the prediction percentage.
        """
        img = image.load_img(file_path, target_size=(128, 128))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        result = self.model.predict(img_array)
        prediction = result[0][0]
        prediction_percentage = prediction * 100
        return prediction, prediction_percentage

    def run(self):
        """
        Run the Flask application with the loaded model.
        """
        self.app.run(debug=True)


if __name__ == '__main__':
    # inference
    model_path = 'deepfake_detector_model.keras'
    inference_model = InferenceModel(model_path)
    inference_model.run()

Writing inference.py


In [5]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Deepfake Detector</title>
    <!-- Bootstrap CSS -->
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container">
        <h1 class="mt-4">Deepfake Detector</h1>

        <form method="POST" enctype="multipart/form-data" class="mt-4">
            <div class="form-group">
                <input type="file" name="file" class="form-control-file">
            </div>
            <button type="submit" class="btn btn-primary">Upload</button>
        </form>

        {% if error %}
            <div class="alert alert-danger mt-4" role="alert">
                {{ error }}
            </div>
        {% endif %}

        {% if result %}
            <div class="alert alert-success mt-4" role="alert">
                <p><strong>Result:</strong> {{ result }}</p>
                <p><strong>Prediction Percentage:</strong> {{ prediction_percentage }}%</p>
            </div>
        {% endif %}
    </div>

    <!-- Bootstrap JS and dependencies -->
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.5.4/dist/umd/popper.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</body>
</html>

Writing templates/index.html


In [6]:
!pip install flask numpy matplotlib opencv-python scikit-learn pandas tqdm requests
# TensorFlow already Colab mein hai, extra install nahi karna
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [ ]:
!python train.py

2026-01-21 18:03:51.043867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769018631.074870     774 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769018631.084285     774 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769018631.106891     774 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769018631.106926     774 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769018631.106934     774 computation_placer.cc:177] computation placer alr

In [ ]:
!cp deepfake_detector_model.keras /content/drive/MyDrive/deepfake_model.keras
print("Model saved to Drive!")

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
# Purane tunnels band karo
!killall ngrok 2>/dev/null
# Flask app background mein chalao
get_ipython().system_raw('python inference.py &')
# Public URL banao
public_url = ngrok.connect(5000)
print("🎉 Browser mein ye URL kholo aur images test karo:")
print(public_url)

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("37bz9imiRzSwkvbctYxFSNeqWJd_7ZCFKvKLb39C4hQKh8u7s")  # quotes ke andar paste karo, quotes hatao mat

In [ ]:
from pyngrok import ngrok
!killall ngrok 2>/dev/null
get_ipython().system_raw('python inference.py &')
public_url = ngrok.connect(5000)
print("🎉 Browser mein ye URL kholo:")
print(public_url)

In [ ]:
!pip install mtcnn opencv-python reportlab tensorflow tensorflow_hub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("debajyatidey/faceforensics-videos-cropped-faces")

print("Path to dataset files:", path)

In [ ]:
!pip install mtcnn opencv-python reportlab

In [ ]:
def download_ffpp_dataset():
    # Download latest version from KaggleHub
    path = kagglehub.dataset_download("debajyatidey/faceforensics-videos-cropped-faces")
    print("Path to dataset files:", path)

    # ⚠️ Do NOT try to create folders inside /kaggle/input (read-only)
    # Just return the dataset path
    return path


In [ ]:
def load_dataset(base_path, split='train', batch_size=32):
    path = os.path.join(base_path, split)
    return tf.keras.utils.image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode='binary',
        batch_size=batch_size,
        image_size=(128, 128),
        seed=42,
        color_mode='rgb'
    )


In [ ]:
import os, shutil, glob
import kagglehub
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import Xception, EfficientNetB0

# === DATASET ===
def download_ffpp_dataset():
    path = kagglehub.dataset_download("debajyatidey/faceforensics-videos-cropped-faces")
    print("Path to dataset files:", path)
    return path

def prepare_dataset(base_path, train_split=0.7, val_split=0.15, test_split=0.15):
    """Split dataset into train/val/test inside /kaggle/working"""
    working_path = "/kaggle/working/ffpp_frames"
    os.makedirs(working_path, exist_ok=True)
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(working_path, split), exist_ok=True)

    # assume dataset has subfolders per class (e.g., 'real', 'fake')
    classes = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    for cls in classes:
        files = glob.glob(os.path.join(base_path, cls, "*.jpg"))
        n = len(files)
        train_end = int(train_split * n)
        val_end = int((train_split + val_split) * n)

        splits = {
            "train": files[:train_end],
            "val": files[train_end:val_end],
            "test": files[val_end:]
        }

        for split, split_files in splits.items():
            cls_dir = os.path.join(working_path, split, cls)
            os.makedirs(cls_dir, exist_ok=True)
            for f in split_files:
                shutil.copy(f, cls_dir)

    return working_path

def load_dataset(base_path, split='train', batch_size=32):
    path = os.path.join(base_path, split)
    return tf.keras.utils.image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode='binary',
        batch_size=batch_size,
        image_size=(128, 128),
        seed=42,
        color_mode='rgb'
    )


In [ ]:
import shutil, glob

def prepare_dataset(base_path, train_split=0.7, val_split=0.15, test_split=0.15):
    """Split dataset into train/val/test inside /kaggle/working"""
    working_path = "/kaggle/working/ffpp_frames"
    os.makedirs(working_path, exist_ok=True)
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(working_path, split), exist_ok=True)

    # assume dataset has subfolders per class (e.g., 'real', 'fake')
    classes = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    for cls in classes:
        files = glob.glob(os.path.join(base_path, cls, "*.jpg"))
        n = len(files)
        train_end = int(train_split * n)
        val_end = int((train_split + val_split) * n)

        splits = {
            "train": files[:train_end],
            "val": files[train_end:val_end],
            "test": files[val_end:]
        }

        for split, split_files in splits.items():
            cls_dir = os.path.join(working_path, split, cls)
            os.makedirs(cls_dir, exist_ok=True)
            for f in split_files:
                shutil.copy(f, cls_dir)

    return working_path


In [ ]:
import os

dataset_path = "/kaggle/input/faceforensics-videos-cropped-faces"
for root, dirs, files in os.walk(dataset_path):
    print(root, dirs[:5], files[:5])  # show first few entries


In [ ]:
import os

dataset_path = "/kaggle/input/faceforensics-videos-cropped-faces"
for root, dirs, files in os.walk(dataset_path):
    if files:
        print(root, files[:5])  # show first 5 files in each folder


In [ ]:
def load_dataset(base_path, split='train', batch_size=32):
    path = os.path.join(base_path, split)
    return tf.keras.utils.image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode='binary',
        batch_size=batch_size,
        image_size=(128, 128),
        seed=42,
        color_mode='rgb'
    )


In [ ]:
%pip install mtcnn[tensorflow]

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os, glob
import cv2
import tensorflow as tf

In [ ]:
base_path = r'/kaggle/input/ff-c23/FaceForensics++_C23/'

ffpp_real = sorted(glob.glob(os.path.join(base_path, 'original', '*.mp4')))
ffpp_fake = sorted(glob.glob(os.path.join(base_path, 'Deepfakes', '*.mp4')))
video_paths = {'real': ffpp_real, 'fake': ffpp_fake}

In [ ]:
print(f'No. of fake videos,- {len(video_paths["fake"])}')
print(f'No. of real videos,- {len(video_paths["real"])}')

In [ ]:
def extract_iframes(video_path, output_dir, num_frames=10):
    """
    Extract equidistant frames from a video using cv2.

    Args:
        video_path (str): Path to input video file.
        output_dir (str): Directory where frames will be saved.
        num_frames (int): Number of frames to extract.
    """
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error opening video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # If requested frames > available, limit it
    num_frames = min(num_frames, total_frames)

    step = total_frames // num_frames

    frame_ids = [i * step for i in range(num_frames)]

    for idx, frame_id in enumerate(frame_ids):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if ret:
            frame_filename = os.path.join(output_dir, f"frame_{idx:04d}.png")
            cv2.imwrite(frame_filename, frame)
        else:
            print(f"Warning: could not read frame {frame_id} in {video_path}")

    cap.release()
    print(f"Extracted {len(frame_ids)} frames from {video_path}")
    frames = sorted(glob.glob(os.path.join(output_dir, "frame_*.png")))
    return frames

In [ ]:
from mtcnn import MTCNN
from mtcnn.utils.images import load_image
def crop_faces_mtcnn(image_paths, output_dir, detector=None):
    """
    Crops the highest-confidence face from each image using MTCNN.
    Returns a list of saved cropped face image paths.
    """
    os.makedirs(output_dir, exist_ok=True)
    if detector is None:
        detector = MTCNN()

    cropped_files = []
    for img_path in image_paths:
        img = load_image(img_path)
        results = detector.detect_faces(img)
        if not results:
            continue

        best_face = max(results, key=lambda x: x['confidence'])
        conf = best_face['confidence']

        x, y, w, h = best_face['box']
        x, y = max(0, x), max(0, y)
        face_crop = img[y:y+h, x:x+w]

        tensor = tf.cast(face_crop, tf.uint8)  # must be uint8 for encoding
        encoded = tf.image.encode_png(tensor)
        out_file = os.path.join(output_dir, os.path.basename(img_path))
        tf.io.write_file(out_file, encoded)
        cropped_files.append(out_file)

    return cropped_files

In [ ]:
def extract_faces_from_video(video_path, work_dir):
    """
    Complete pipeline: extracts Keyframes from a video, then crops faces with MTCNN.
    Returns list of cropped face image paths.
    """
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    frames_dir = os.path.join(work_dir, f"{base_name}_frames")
    crops_dir = os.path.join(work_dir, f"{base_name}_crops")

    # Step 1: Extract Keyframes
    frames = extract_iframes(video_path, frames_dir)

    # Step 2: Crop faces
    detector = MTCNN()
    cropped_faces = crop_faces_mtcnn(frames, crops_dir, detector=detector)

    return cropped_faces

In [ ]:
import gc
for label in video_paths:
    print(f'\nExtracting and Cropping {label} videos\n')
    out_dir = f"/kaggle/working/{label}/"
    for video_path in video_paths[label]:
        cropped_faces = extract_faces_from_video(video_path,out_dir)
        print(f"Extracted and cropped {len(cropped_faces)} faces from {video_path}")
        gc.collect()
        !rm -rf /kaggle/working/*/*_frames/

In [ ]:
!pip install mtcnn opencv-python reportlab tensorflow tensorflow_hub kagglehub

In [ ]:
# %%writefile train.py
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import Xception, EfficientNetB0

# === DATASET: FaceForensics++ Cropped Faces (Public Kaggle Dataset) ===
def download_ffpp_cropped_dataset():
    # Download the dataset using kagglehub
    print("Downloading FF++ Cropped Faces dataset...")
    path = kagglehub.dataset_download("debajyatidey/faceforensics-videos-cropped-faces")
    print(f"Dataset downloaded to: {path}")
    return path

def load_dataset(dataset_path, split='train', batch_size=32):
    """Load dataset from the specified directory."""
    path = os.path.join(dataset_path, split)
    return tf.keras.utils.image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode='binary',
        batch_size=batch_size,
        image_size=(128, 128),
        seed=42,
        color_mode='rgb'
    )

# === MODELS ===
def build_custom_cnn():
    model = models.Sequential([
        layers.Input((128, 128, 3)),
        layers.Rescaling(1./127.5, offset=-1),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_xception():
    base = Xception(weights='imagenet', include_top=False, input_shape=(128,128,3))
    base.trainable = False
    model = models.Sequential([
        layers.Rescaling(1./127.5, offset=-1),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_efficientnet():
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128,128,3))
    base.trainable = False
    model = models.Sequential([
        layers.Rescaling(1./127.5, offset=-1),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# === TRAINING ===
def train_model(model, name, train_ds, val_ds, epochs=20):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(f"{name}_best.keras", save_best_only=True)
    ]
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)
    model.save(f"{name}.keras")
    return history

if __name__ == "__main__":
    # Authenticate Kaggle (using your provided key)
    try:
        from google.colab import files
        # Create kaggle.json with your key
        with open('kaggle.json', 'w') as f:
            f.write('{"username":"your_username","key":"KGAT_3ac23225afad9972c5f6d2cc6bc41220"}')
        !mkdir -p ~/.kaggle
        !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        print("Kaggle authenticated successfully.")
    except Exception as e:
        print(f"Kaggle authentication failed: {e}")

    # Download FF++ dataset
    dataset_path = download_ffpp_cropped_dataset()

    # Load datasets
    train_ds = load_dataset(dataset_path, 'train', 32).prefetch(tf.data.AUTOTUNE)
    val_ds = load_dataset(dataset_path, 'val', 32).prefetch(tf.data.AUTOTUNE)
    test_ds = load_dataset(dataset_path, 'test', 32).prefetch(tf.data.AUTOTUNE)

    # Train models
    for model_fn, name in [(build_custom_cnn, "custom_cnn"),
                           (build_xception, "xception"),
                           (build_efficientnet, "efficientnet")]:
        print(f"\nTraining {name}...")
        model = model_fn()
        train_model(model, name, train_ds, val_ds, epochs=15)

In [ ]:
!ls /kaggle/input/faceforensics-videos-cropped-faces

In [ ]:
# %%writefile train.py
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import Xception, EfficientNetB0
from sklearn.model_selection import train_test_split
import numpy as np

# === DATASET: FaceForensics++ Cropped Faces ===
def download_ffpp_cropped_dataset():
    print("Downloading FF++ Cropped Faces dataset...")
    path = kagglehub.dataset_download("debajyatidey/faceforensics-videos-cropped-faces")
    print(f"Dataset downloaded to: {path}")
    return path

def load_and_split_dataset(dataset_path, batch_size=32, val_split=0.15, test_split=0.15):
    """
    Load the entire dataset and split it into train/val/test.
    Assumes root directory has 'real/' and 'fake/' subfolders.
    """
    full_ds = tf.keras.utils.image_dataset_from_directory(
        dataset_path,
        labels='inferred',
        label_mode='binary',
        batch_size=None,  # No batching yet
        image_size=(128, 128),
        seed=42,
        color_mode='rgb'
    )

    # Convert to lists for splitting
    images, labels = [], []
    for img, lbl in full_ds:
        images.append(img.numpy())
        labels.append(lbl.numpy())

    images = np.array(images)
    labels = np.array(labels)

    # First split: train + (val + test)
    X_train, X_temp, y_train, y_temp = train_test_split(
        images, labels, test_size=(val_split + test_split), random_state=42, stratify=labels
    )

    # Second split: val and test
    val_ratio = val_split / (val_split + test_split)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio), random_state=42, stratify=y_temp
    )

    # Convert back to tf.data.Dataset
    def make_dataset(X, y, batch_size):
        ds = tf.data.Dataset.from_tensor_slices((X, y))
        return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    train_ds = make_dataset(X_train, y_train, batch_size)
    val_ds = make_dataset(X_val, y_val, batch_size)
    test_ds = make_dataset(X_test, y_test, batch_size)

    return train_ds, val_ds, test_ds

# === MODELS ===
def build_custom_cnn():
    model = models.Sequential([
        layers.Input((128, 128, 3)),
        layers.Rescaling(1./127.5, offset=-1),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_xception():
    base = Xception(weights='imagenet', include_top=False, input_shape=(128,128,3))
    base.trainable = False
    model = models.Sequential([
        layers.Rescaling(1./127.5, offset=-1),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_efficientnet():
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128,128,3))
    base.trainable = False
    model = models.Sequential([
        layers.Rescaling(1./127.5, offset=-1),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# === TRAINING ===
def train_model(model, name, train_ds, val_ds, epochs=20):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(f"{name}_best.keras", save_best_only=True)
    ]
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)
    model.save(f"{name}.keras")
    return history

if __name__ == "__main__":
    # Authenticate Kaggle
    try:
        with open('kaggle.json', 'w') as f:
            f.write('{"username":"your_username","key":"KGAT_3ac23225afad9972c5f6d2cc6bc41220"}')
        !mkdir -p ~/.kaggle
        !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        print("Kaggle authenticated successfully.")
    except Exception as e:
        print(f"Kaggle authentication skipped: {e}")

    # Download FF++ dataset
    dataset_path = download_ffpp_cropped_dataset()

    # Load and split dataset
    print("Loading and splitting dataset...")
    train_ds, val_ds, test_ds = load_and_split_dataset(dataset_path, batch_size=32)

    # Train models
    for model_fn, name in [(build_custom_cnn, "custom_cnn"),
                           (build_xception, "xception"),
                           (build_efficientnet, "efficientnet")]:
        print(f"\nTraining {name}...")
        model = model_fn()
        train_model(model, name, train_ds, val_ds, epochs=15)

In [ ]:
# %%writefile autonomous_agent.py
import os
import cv2
import numpy as np
import json
from datetime import datetime
from mtcnn import MTCNN
from tensorflow.keras.models import load_model
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Initialize MTCNN
detector = MTCNN()

def extract_frames(video_path, max_frames=30):
    """Extract up to `max_frames` from a video."""
    cap = cv2.VideoCapture(video_path)
    frames = []
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total == 0:
        cap.release()
        return frames  # Empty video
    step = max(1, total // max_frames)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % step == 0:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        count += 1
    cap.release()
    return frames[:max_frames]

def detect_and_crop_face(img):
    """Detect the largest face in an image and crop it."""
    faces = detector.detect_faces(img)
    if not faces:
        return None
    # Sort by face area (largest first)
    faces = sorted(faces, key=lambda x: x['box'][2] * x['box'][3], reverse=True)
    x, y, w, h = faces[0]['box']
    x, y = max(0, x), max(0, y)
    cropped = img[y:y+h, x:x+w]
    return cv2.resize(cropped, (128, 128))

def load_models():
    """Load all trained models."""
    models = {}
    for name in ["custom_cnn", "xception", "efficientnet"]:
        model_path = f"{name}_best.keras"
        if os.path.exists(model_path):
            try:
                models[name] = load_model(model_path)
                print(f"Loaded model: {name}")
            except Exception as e:
                print(f"Failed to load {name}: {e}")
    return models

def predict_on_image(img, models):
    """Predict on a single image using the best available model."""
    img = np.expand_dims(img.astype('float32') / 127.5 - 1, axis=0)
    results = {}
    for name, model in models.items():
        pred = model.predict(img, verbose=0)[0][0]
        results[name] = float(pred)

    # Select the best model (Xception preferred for FF++)
    if "xception" in results:
        final = results["xception"]
        model_used = "Xception"
    elif "efficientnet" in results:
        final = results["efficientnet"]
        model_used = "EfficientNet-B0"
    elif "custom_cnn" in results:
        final = results["custom_cnn"]
        model_used = "Custom CNN"
    else:
        raise ValueError("No valid models loaded!")

    return final, model_used, results

def generate_pdf_report(report_data, pdf_path):
    """Generate a forensic PDF report with Colab-safe fonts."""
    try:
        # Register a standard font to avoid Colab issues
        from reportlab.pdfbase.pdfmetrics import registerFont
        from reportlab.pdfbase.ttfonts import TTFont
        # Use a built-in font
        c = canvas.Canvas(pdf_path, pagesize=letter)

        # Title
        c.setFont("Helvetica-Bold", 16)
        c.drawString(100, 750, "Deepfake Forensic Report")

        # Details
        c.setFont("Helvetica", 12)
        c.drawString(100, 730, f"File: {report_data.get('filename', 'N/A')}")
        c.drawString(100, 710, f"Result: {'FAKE' if report_data['is_fake'] else 'REAL'}")
        c.drawString(100, 690, f"Confidence: {report_data['confidence']:.2%}")
        c.drawString(100, 670, f"Model Used: {report_data['model_used']}")
        c.drawString(100, 650, f"Analysis Time: {report_data['timestamp']}")

        if 'frame_count' in report_data:
            c.drawString(100, 630, f"Frames Analyzed: {report_data['frame_count']}")
            c.drawString(100, 610, f"Frame Confidence Std Dev: {report_data.get('std_dev', 0):.4f}")

        c.save()
        print(f"PDF report saved to: {pdf_path}")
    except Exception as e:
        print(f"PDF generation failed: {e}")
        # Fallback: just create an empty file to avoid crashing
        with open(pdf_path, 'w') as f:
            f.write("PDF generation failed due to Colab environment limitations.")

def run_agent(file_path):
    """Main function to run the autonomous agent."""
    is_video = file_path.lower().endswith(('.mp4', '.avi', '.mov', '.webm'))
    models = load_models()

    if not models:
        return {"error": "No trained models found. Please run train.py first."}

    if is_video:
        print("Processing video...")
        frames = extract_frames(file_path)
        if not frames:
            return {"error": "No frames extracted from video. Invalid or corrupted file."}

        predictions = []
        for i, frame in enumerate(frames):
            face = detect_and_crop_face(frame)
            if face is not None:
                pred, model_used, _ = predict_on_image(face, models)
                predictions.append(pred)
            # Optional: Add progress indicator
            if (i + 1) % 10 == 0:
                print(f"Processed {i+1}/{len(frames)} frames...")

        if not predictions:
            return {"error": "No faces detected in video"}

        avg_conf = np.mean(predictions)
        std_conf = np.std(predictions)
        is_fake = avg_conf >= 0.5

        result = {
            "file_type": "video",
            "filename": os.path.basename(file_path),
            "model_used": model_used,
            "is_fake": bool(is_fake),
            "confidence": float(avg_conf),
            "std_dev": float(std_conf),
            "frame_count": len(predictions),
            "frame_predictions": [float(p) for p in predictions],
            "timestamp": datetime.now().isoformat()
        }
    else:
        print("Processing image...")
        img = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        if img is None:
            return {"error": "Invalid image file. Could not load."}

        face = detect_and_crop_face(img)
        if face is None:
            return {"error": "No face detected in image"}

        pred, model_used, all_preds = predict_on_image(face, models)
        result = {
            "file_type": "image",
            "filename": os.path.basename(file_path),
            "model_used": model_used,
            "is_fake": bool(pred >= 0.5),
            "confidence": float(pred),
            "all_model_predictions": all_preds,
            "timestamp": datetime.now().isoformat()
        }

    # Generate JSON report
    json_path = file_path + "_report.json"
    with open(json_path, "w") as f:
        json.dump(result, f, indent=2)

    # Generate PDF report
    pdf_path = file_path + "_report.pdf"
    generate_pdf_report(result, pdf_path)

    result["json_report"] = json_path
    result["pdf_report"] = pdf_path
    return result

In [ ]:
!pip install mtcnn opencv-python reportlab tensorflow_hub

In [ ]:
# %%writefile train_additional_models.py
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import Xception, EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load your existing dataset (same structure as your current train.py)
def load_split_data():
    def load_dir(name):
        return tf.keras.utils.image_dataset_from_directory(
            f'./data/Dataset/{name}',
            labels='inferred',
            label_mode='binary',
            batch_size=32,
            image_size=(128, 128),
            seed=42
        )
    return load_dir('Train'), load_dir('Validation'), load_dir('Test')

def build_xception():
    base = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base.trainable = False
    return models.Sequential([
        layers.Rescaling(1./127.5, offset=-1),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

def build_efficientnet():
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base.trainable = False
    return models.Sequential([
        layers.Rescaling(1./127.5, offset=-1),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

def train_and_save(model_fn, name):
    print(f"\nTraining {name}...")
    train, val, _ = load_split_data()
    model = model_fn()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(f'{name}_best.keras', save_best_only=True)
    ]
    model.fit(train, validation_data=val, epochs=20, callbacks=callbacks)
    model.save(f'{name}.keras')

if __name__ == "__main__":
    train_and_save(build_xception, "xception")
    train_and_save(build_efficientnet, "efficientnet")
    print("\n✅ Xception & EfficientNet training complete!")

In [ ]:
# %%writefile autonomous_agent.py
import os
import cv2
import numpy as np
import json
from datetime import datetime
from mtcnn import MTCNN
from tensorflow.keras.models import load_model
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

detector = MTCNN()

# --- Helper Functions ---
def extract_frames(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames, total = [], int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total == 0: return frames
    step = max(1, total // max_frames)
    for i in range(total):
        ret, frame = cap.read()
        if not ret: break
        if i % step == 0:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if len(frames) >= max_frames: break
    cap.release()
    return frames

def detect_and_crop_face(img):
    faces = detector.detect_faces(img)
    if not faces: return None
    x, y, w, h = max(faces, key=lambda f: f['box'][2]*f['box'][3])['box']
    x, y = max(0, x), max(0, y)
    return cv2.resize(img[y:y+h, x:x+w], (128, 128))

def load_all_models():
    models = {}
    # Try loading proposal-compliant models first
    for name in ["xception", "efficientnet"]:
        if os.path.exists(f"{name}_best.keras"):
            models[name] = load_model(f"{name}_best.keras")
    # Fallback to your original model
    if not models and os.path.exists("deepfake_detector_model.keras"):
        models["custom_cnn"] = load_model("deepfake_detector_model.keras")
    return models

def predict_image(img, models):
    img = np.expand_dims(img.astype('float32') / 127.5 - 1, axis=0)
    preds = {name: float(model.predict(img, verbose=0)[0][0]) for name, model in models.items()}
    # Select best model (proposal priority: Xception > EfficientNet > Custom)
    if "xception" in preds:
        final, model_used = preds["xception"], "Xception"
    elif "efficientnet" in preds:
        final, model_used = preds["efficientnet"], "EfficientNet-B0"
    else:
        final, model_used = preds["custom_cnn"], "Custom CNN"
    return final, model_used, preds

def generate_pdf_report(data, path):
    c = canvas.Canvas(path, pagesize=letter)
    c.drawString(100, 750, "Deepfake Forensic Report")
    c.drawString(100, 730, f"File: {data['filename']}")
    c.drawString(100, 710, f"Result: {'FAKE' if data['is_fake'] else 'REAL'}")
    c.drawString(100, 690, f"Confidence: {data['confidence']:.2%}")
    c.drawString(100, 670, f"Model Used: {data['model_used']}")
    c.save()

# --- Main Agent Function ---
def run_autonomous_agent(file_path):
    is_video = file_path.lower().endswith(('.mp4', '.avi', '.mov'))
    models = load_all_models()
    if not models:
        return {"error": "No models found!"}

    if is_video:
        frames = extract_frames(file_path)
        preds = []
        for frame in frames:
            face = detect_and_crop_face(frame)
            if face is not None:
                pred, model_used, _ = predict_image(face, models)
                preds.append(pred)
        if not preds:
            return {"error": "No faces detected in video"}
        avg_conf = np.mean(preds)
        result = {
            "file_type": "video",
            "filename": os.path.basename(file_path),
            "model_used": model_used,
            "is_fake": bool(avg_conf >= 0.5),
            "confidence": float(avg_conf),
            "frame_count": len(preds),
            "std_dev": float(np.std(preds))
        }
    else:
        img = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        face = detect_and_crop_face(img)
        if face is None:
            return {"error": "No face detected in image"}
        pred, model_used, all_preds = predict_image(face, models)
        result = {
            "file_type": "image",
            "filename": os.path.basename(file_path),
            "model_used": model_used,
            "is_fake": bool(pred >= 0.5),
            "confidence": float(pred)
        }

    # Save reports
    json_path = file_path + "_report.json"
    pdf_path = file_path + "_report.pdf"
    with open(json_path, "w") as f:
        json.dump(result, f, indent=2)
    generate_pdf_report(result, pdf_path)
    result.update({"json_report": json_path, "pdf_report": pdf_path})
    return result

In [ ]:
# %%writefile inference.py
from flask import Flask, request, render_template, send_file
import os
import tempfile
from autonomous_agent import run_autonomous_agent

app = Flask(__name__)
os.makedirs("uploads", exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files.get('file')
        if not file or file.filename == '':
            return render_template('index.html', error='No file selected')
        if not file.filename.lower().endswith(('.png', '.jpg', '.jpeg', '.mp4', '.avi', '.mov')):
            return render_template('index.html', error='Invalid file type (use JPG/PNG/MP4/AVI)')

        temp_path = os.path.join(tempfile.gettempdir(), file.filename)
        file.save(temp_path)

        try:
            result = run_autonomous_agent(temp_path)
            os.remove(temp_path)
            if "error" in result:
                return render_template('index.html', error=result["error"])
            return render_template('index.html',
                result="Fake" if result["is_fake"] else "Real",
                confidence=f"{result['confidence']:.2%}",
                model=result["model_used"],
                pdf_report=os.path.basename(result["pdf_report"])
            )
        except Exception as e:
            return render_template('index.html', error=f"Processing failed: {str(e)}")
    return render_template('index.html')

@app.route('/download/<filename>')
def download_report(filename):
    return send_file(os.path.join(tempfile.gettempdir(), filename), as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
<!-- templates/index.html -->
<!DOCTYPE html>
<html>
<head>
  <title>Autonomous Deepfake Detector</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
<div class="container mt-4">
  <h2>Autonomous Deepfake Forensic Agent</h2>
  <form method="POST" enctype="multipart/form-data">
    <div class="mb-3">
      <input class="form-control" type="file" name="file" accept=".jpg,.jpeg,.png,.mp4,.avi,.mov" required>
    </div>
    <button class="btn btn-primary" type="submit">Analyze</button>
  </form>

  {% if error %}
    <div class="alert alert-danger mt-3">{{ error }}</div>
  {% endif %}

  {% if result %}
    <div class="card mt-3">
      <div class="card-body">
        <h5 class="card-title">Result: <span class="badge bg-{{ 'danger' if result=='Fake' else 'success' }}">{{ result }}</span></h5>
        <p>Confidence: <strong>{{ confidence }}</strong></p>
        <p>Model Used: {{ model }}</p>
        <a href="/download/{{ pdf_report }}" class="btn btn-outline-secondary">Download PDF Report</a>
      </div>
    </div>
  {% endif %}
</div>
</body>
</html>

In [ ]:
!killall ngrok 2>/dev/null
get_ipython().system_raw('python inference.py &')

from pyngrok import ngrok
ngrok.set_auth_token("37bz9imiRzSwkvbctYxFSNeqWJd_7ZCFKvKLb39C4hQKh8u7s")
public_url = ngrok.connect(5000)
print(f"🌐 Public URL: {public_url}")